In [1]:
%cd ~/Downloads

/home/x/Downloads


In [ ]:
! wget https://github.com/NVIDIA/TorchFort/archive/refs/tags/v0.3.0.tar.gz

In [ ]:
! tar zxvf v0.3.0.tar.gz

In [ ]:
! sudo mkdir /torchfort && sudo chown x:x /torchfort

In [ ]:
! cp -r TorchFort-0.3.0/* /torchfort

In [5]:
! sudo docker image ls

REPOSITORY    TAG                       IMAGE ID       CREATED         SIZE
torchfort     latest                    82ff5885af72   7 days ago      37.2GB
torchfort     v4                        80e7a7a5bfcd   7 weeks ago     32GB
torchfort     v3                        588b0fba0bad   2 months ago    32GB
torchfort     v2                        5ef57bf9de13   2 months ago    30.3GB
torchfort     v0                        f4a1749ea5e5   2 months ago    28.9GB
nvidia/cuda   12.3.1-base-ubuntu22.04   bcdbb14063fa   18 months ago   243MB


In [6]:
%cd /torchfort

/torchfort


"rm -rf torchfort" removed:

In [ ]:
%%writefile /torchfort/docker/Dockerfile
FROM nvcr.io/nvidia/cuda:12.8.1-devel-ubuntu22.04

# Install System Dependencies
ENV DEBIAN_FRONTEND noninteractive
RUN apt update -y && \
    apt install -y curl unzip wget cmake && \
    apt install -y python3 python-is-python3 python3-pip python3-pybind11 && \
    apt install -y git vim gfortran doxygen && \
    apt install -y libibverbs-dev ibverbs-utils numactl

# Install NVHPC SDK
RUN wget https://developer.download.nvidia.com/hpc-sdk/25.3/nvhpc_2025_253_Linux_x86_64_cuda_12.8.tar.gz && \
    tar xpzf nvhpc_2025_253_Linux_x86_64_cuda_12.8.tar.gz && \
    nvhpc_2025_253_Linux_x86_64_cuda_12.8/install --quiet && \
    rm -rf nvhpc_2025_253_Linux_x86_64_cuda_12.8 nvhpc_2025_253_Linux_x86_64_cuda_12.8.tar.gz

ENV PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/compilers/bin:$PATH
ENV PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/comm_libs/mpi/bin:$PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda/lib64:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/comm_libs/mpi/lib:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/comm_libs/nvshmem/lib:$LD_LIBRARY_PATH
ENV LD_LIBRARY_PATH /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/math_libs/lib64:$LD_LIBRARY_PATH
ENV CUDA_HOME /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda

RUN echo "source /opt/nvidia/hpc_sdk/Linux_x86_64/25.3/comm_libs/12.8/hpcx/latest/hpcx-init.sh; hpcx_load" >>  /root/.bashrc

# Install PyTorch
RUN pip3 install torch==2.7.0

# Install yaml-cpp
RUN git clone https://github.com/jbeder/yaml-cpp.git --branch 0.8.0 && \
    cd yaml-cpp && \
    mkdir build && cd build && \
    cmake -DCMAKE_INSTALL_PREFIX=/opt/yaml-cpp \
          -DCMAKE_CXX_FLAGS="-D_GLIBCXX_USE_CXX11_ABI=1" \
          -DBUILD_SHARED_LIBS=OFF \
          -DCMAKE_POSITION_INDEPENDENT_CODE=ON .. && \
    make -j$(nproc) && make install
ENV LD_LIBRARY_PATH /opt/yaml-cpp/lib:${LD_LIBRARY_PATH}

# Install HDF5
RUN wget https://github.com/HDFGroup/hdf5/archive/refs/tags/hdf5-1_14_3.tar.gz && \
    tar xzf hdf5-1_14_3.tar.gz && \
    cd hdf5-hdf5-1_14_3 && \
    CC=mpicc FC=mpifort FCFLAGS=-fPIC CFLAGS=-fPIC \
    ./configure --enable-parallel \
                --enable-fortran \
                --prefix=/opt/hdf5 && \
    make -j$(nproc) install && \
    cd .. && \
    rm -rf hdf5-hdf5-1_14_3 hdf5-1_14_3.tar.gz
ENV LD_LIBRARY_PATH /opt/hdf5/lib:$LD_LIBRARY_PATH

# Install additional Python dependencies
RUN pip3 install wandb ruamel-yaml h5py matplotlib pygame moviepy

# Install TorchFort
ENV FC=nvfortran
ENV HDF5_ROOT=/opt/hdf5
COPY . /torchfort
RUN cd /torchfort && mkdir build && cd build && \
    CUDA_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/25.3/cuda \
    cmake -DCMAKE_INSTALL_PREFIX=/opt/torchfort \
    -DCMAKE_CXX_COMPILER=`which g++` \
    -DTORCHFORT_YAML_CPP_ROOT=/opt/yaml-cpp \
    -DTORCHFORT_NCCL_ROOT=/opt/nccl/build \
    -DTORCHFORT_BUILD_EXAMPLES=1 \
    -DTORCHFORT_BUILD_TESTS=1 \
    -DCMAKE_PREFIX_PATH="`python -c 'import torch;print(torch.utils.cmake_prefix_path)'`" \
    .. && \
    make -j$(nproc) install && \
    cd /
ENV LD_LIBRARY_PATH /opt/torchfort/lib:${LD_LIBRARY_PATH}
ENV LD_LIBRARY_PATH /usr/local/lib/python3.10/dist-packages/torch/lib:${LD_LIBRARY_PATH}

ENTRYPOINT bash


In [7]:
! sudo docker build -t torchfort:test01 -f docker/Dockerfile .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 3.26kB                                     0.0s
 => [internal] load metadata for nvcr.io/nvidia/cuda:12.8.1-devel-ubuntu2  0.2s
[+] Building 0.3s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 3.26kB                                     0.0s
 => [internal] load metadata for nvcr.io/nvidia/cuda:12.8.1-devel-ubuntu2  0.3s
[+] Building 0.5s (1/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 3.26kB                                     0.0s
 => [internal] load metadata for nvcr.io

- Convert to .sif. See: dock-to-singu.ipynb

In [9]:
! sudo docker image ls

REPOSITORY    TAG                       IMAGE ID       CREATED          SIZE
torchfort     test01                    7faba7b5e678   20 minutes ago   37.9GB
torchfort     latest                    82ff5885af72   7 days ago       37.2GB
torchfort     v4                        80e7a7a5bfcd   7 weeks ago      32GB
torchfort     v3                        588b0fba0bad   2 months ago     32GB
torchfort     v2                        5ef57bf9de13   2 months ago     30.3GB
torchfort     v0                        f4a1749ea5e5   2 months ago     28.9GB
nvidia/cuda   12.3.1-base-ubuntu22.04   bcdbb14063fa   18 months ago    243MB


In [5]:
! sudo docker rmi torchfort:test01

Untagged: torchfort:test01


In [6]:
! sudo docker images

REPOSITORY    TAG                       IMAGE ID       CREATED         SIZE
torchfort     latest                    82ff5885af72   3 days ago      37.2GB
torchfort     v4                        80e7a7a5bfcd   6 weeks ago     32GB
torchfort     v3                        588b0fba0bad   2 months ago    32GB
torchfort     v2                        5ef57bf9de13   2 months ago    30.3GB
torchfort     v0                        f4a1749ea5e5   2 months ago    28.9GB
nvidia/cuda   12.3.1-base-ubuntu22.04   bcdbb14063fa   18 months ago   243MB


In [2]:
! sudo docker images

REPOSITORY    TAG                       IMAGE ID       CREATED         SIZE
torchfort     latest                    82ff5885af72   12 hours ago    37.2GB
torchfort     v4                        80e7a7a5bfcd   6 weeks ago     32GB
torchfort     v3                        588b0fba0bad   8 weeks ago     32GB
torchfort     v2                        5ef57bf9de13   2 months ago    30.3GB
torchfort     v0                        f4a1749ea5e5   2 months ago    28.9GB
nvidia/cuda   12.3.1-base-ubuntu22.04   bcdbb14063fa   18 months ago   243MB


In [3]:
! sudo singularity build torchfort250619.sif docker-daemon://torchfort:latest

INFO:    Starting build...
INFO:    Fetching OCI image...
INFO:    Extracting OCI image...
INFO:    Inserting Singularity configuration...
INFO:    Creating SIF file...
INFO:    Build complete: torchfort250619.sif


In [4]:
! mv torchfort250619.sif $HOME/containers/

In [6]:
TFT="~/containers/torchfort250619.sif"

In [7]:
! singularity exec --nv {TFT} cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.5 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.5 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


## rsync

In [1]:
%%bash
export SCR=$(ssh -T sd 'echo "/scratch${HOME#/prj}"')
rsync \
    --delete \
    --recursive \
    --inplace \
    --no-links \
    --no-perms \
    --omit-dir-times \
    --times \
    --modify-window=2 \
    --timeout=60 \
    --info=progress2 \
    --human-readable \
    ~/sd-scr-sync/torchfort.sif \
    sd:$SCR/containers

         16,00G 100%    9,88MB/s    0:25:44 (xfr#1, to-chk=0/1)
